In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import random
import time
import pandas as pd
import webdataset as wds
from tqdm.auto import tqdm
import subprocess
import csv 
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score
from xgboost import XGBClassifier
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# from torchdata.datapipes.iter import FileLister, FileOpener
# from functools import partial
# import torch
# from torch.utils.data import Dataset, DataLoader
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim

# 1. Construct the multiomics dataset

## 1(a). First, add the miRNA + gene expression dataset (primary tumor + normal samples)

### Summary dataframe

In [2]:
df = pd.read_csv("subset_primarytumorandnormal_with_omics_types.csv", index_col = 0)
df

,Sample,miRNA,gene_exp,snp,meth,n_omics_types,type,meth27k_dataset,meth450k_dataset
0,TCGA-D8-A146-01,1,1,1,1,4,Primary Tumor,df_BRCA_meth27,NaN
1,TCGA-AQ-A0Y5-01,1,1,1,1,4,Primary Tumor,NaN,df_BRCA_meth450
2,TCGA-C8-A1HI-01,1,1,1,1,4,Primary Tumor,NaN,df_BRCA_meth450
3,TCGA-A7-A0CD-01,1,1,1,1,4,Primary Tumor,df_BRCA_meth27,NaN
4,TCGA-5L-AAT1-01,1,1,1,1,4,Primary Tumor,NaN,df_BRCA_meth450
...,...,...,...,...,...,...,...,...,...
11886,TCGA-E2-A1IO-11,0,0,0,1,1,Solid Tissue Normal,NaN,df_BRCA_meth450
11887,TCGA-BH-A0BF-11,0,0,0,1,1,Solid Tissue Normal,NaN,df_BRCA_meth450
11888,TCGA-E2-A1IF-11,0,0,0,1,1,Solid Tissue Normal,NaN,df_BRCA_meth450
11889,TCGA-BH-A1EY-11,0,0,0,1,1,Solid Tissue Normal,NaN,df_BRCA_meth450


In [3]:
df_subset = df[(df.miRNA == 1) & (df.gene_exp == 1) & (df.meth450k_dataset.notna())] # get samples with all 3 omics: gene expression + miRNA + methylation450k
df_subset.reset_index(inplace=True, drop=True)
df_subset

,Sample,miRNA,gene_exp,snp,meth,n_omics_types,type,meth27k_dataset,meth450k_dataset
0,TCGA-AQ-A0Y5-01,1,1,1,1,4,Primary Tumor,NaN,df_BRCA_meth450
1,TCGA-C8-A1HI-01,1,1,1,1,4,Primary Tumor,NaN,df_BRCA_meth450
2,TCGA-5L-AAT1-01,1,1,1,1,4,Primary Tumor,NaN,df_BRCA_meth450
3,TCGA-BH-A0C0-01,1,1,1,1,4,Primary Tumor,NaN,df_BRCA_meth450
4,TCGA-B6-A1KC-01,1,1,1,1,4,Primary Tumor,NaN,df_BRCA_meth450
...,...,...,...,...,...,...,...,...,...
8462,TCGA-W5-AA31-01,1,1,1,1,4,Primary Tumor,NaN,df_CHOL_meth450
8463,TCGA-W5-AA2U-01,1,1,1,1,4,Primary Tumor,NaN,df_CHOL_meth450
8464,TCGA-ZU-A8S4-01,1,1,1,1,4,Primary Tumor,NaN,df_CHOL_meth450
8465,TCGA-W5-AA2I-01,1,1,1,1,4,Primary Tumor,NaN,df_CHOL_meth450


### Gene expression data

In [4]:
start_time = time.time()
df_gene_exp_full = pd.read_csv("for_yu_gene_expression.csv")
print((time.time() - start_time)/60)
df_gene_exp_full

5.780484016736349


,Unnamed: 0,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,...,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1,type,stage,barcode
0,TCGA-BRCA1,5.662037,3.376096,6.860140,4.400552,2.845169,3.579965,4.996181,5.614683,4.385652,...,0,0.000000,0.000000,3.533999,0,0.058109,1.122739,Primary Tumor,Stage IIA,TCGA-D8-A146-01A-31R-A115-07
1,TCGA-BRCA2,3.703721,0.463099,7.086452,4.051007,2.426989,2.322361,4.943860,6.539078,4.046404,...,0,0.000000,0.000000,4.773448,0,0.054779,1.243182,Primary Tumor,Stage IIIA,TCGA-AQ-A0Y5-01A-11R-A14M-07
2,TCGA-BRCA3,6.514515,0.000000,6.805072,5.037264,4.043248,2.124130,2.993348,5.119912,3.409907,...,0,0.000000,0.000000,4.701710,0,0.093425,0.604261,Primary Tumor,Stage IIA,TCGA-C8-A274-01A-11R-A16F-07
3,TCGA-BRCA4,4.784917,2.328061,6.443071,4.374970,4.162790,3.122375,4.754674,5.608738,3.625691,...,0,0.966357,0.000000,4.689075,0,0.059771,0.536351,Primary Tumor,Stage IIA,TCGA-BH-A0BD-01A-11R-A034-07
4,TCGA-BRCA5,4.251984,0.435415,6.178436,3.558464,2.589548,1.508581,3.722149,5.224044,3.242221,...,0,0.000000,0.000000,5.168221,0,0.023752,0.389126,Primary Tumor,Stage IIB,TCGA-B6-A1KC-01B-11R-A157-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10814,TCGA-CHOL40,5.312015,0.054779,4.956693,1.847115,0.827900,2.303225,8.427157,4.806525,4.081399,...,0,0.000000,0.263756,2.721504,0,0.008487,0.422233,Solid Tissue Normal,Stage I,TCGA-W5-AA30-11A-11R-A41I-07
10815,TCGA-CHOL41,7.072792,0.070252,6.705809,2.356285,1.864176,2.305241,2.342270,6.360315,4.059762,...,0,0.000000,0.000000,3.773828,0,0.032806,1.390888,Primary Tumor,Stage I,TCGA-W5-AA38-01A-11R-A41I-07
10816,TCGA-CHOL42,2.132873,0.298893,6.025226,2.472904,1.310224,4.988394,3.466940,4.420954,3.570706,...,0,1.080044,0.095182,2.120053,0,0.040261,1.388465,Primary Tumor,Stage III,TCGA-W5-AA2H-01A-31R-A41I-07
10817,TCGA-CHOL43,5.638529,0.000000,6.587676,3.354960,2.290601,2.730901,6.821506,6.342717,3.480162,...,0,0.000000,0.054779,4.356369,0,0.132643,1.277747,Primary Tumor,Stage I,TCGA-W5-AA2O-01A-11R-A41I-07


In [5]:
for col in df_gene_exp_full.columns:
    if "ENSG" not in col:
        print(col)

Unnamed: 0
type
stage
barcode


In [6]:
df_gene_exp_full.drop(columns = ['Unnamed: 0', 'type', 'stage'], inplace = True)

In [7]:
# extract sample-level identifier (substring of the full barcode/sample ID)
df_gene_exp_full['sample_id'] = df_gene_exp_full.barcode.str.split('-', n=4).str[:4].str.join('-').str[:-1]
df_gene_exp_full = df_gene_exp_full.drop(columns = ['barcode'])
df_gene_exp_full

,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,ENSG00000001167.14,...,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1,sample_id
0,5.662037,3.376096,6.860140,4.400552,2.845169,3.579965,4.996181,5.614683,4.385652,5.843705,...,0.00000,0.537346,0,0.000000,0.000000,3.533999,0,0.058109,1.122739,TCGA-D8-A146-01
1,3.703721,0.463099,7.086452,4.051007,2.426989,2.322361,4.943860,6.539078,4.046404,5.711726,...,0.00000,0.204767,0,0.000000,0.000000,4.773448,0,0.054779,1.243182,TCGA-AQ-A0Y5-01
2,6.514515,0.000000,6.805072,5.037264,4.043248,2.124130,2.993348,5.119912,3.409907,5.963742,...,0.00000,0.412402,0,0.000000,0.000000,4.701710,0,0.093425,0.604261,TCGA-C8-A274-01
3,4.784917,2.328061,6.443071,4.374970,4.162790,3.122375,4.754674,5.608738,3.625691,5.678199,...,0.00000,0.268075,0,0.966357,0.000000,4.689075,0,0.059771,0.536351,TCGA-BH-A0BD-01
4,4.251984,0.435415,6.178436,3.558464,2.589548,1.508581,3.722149,5.224044,3.242221,5.956145,...,0.00000,0.434028,0,0.000000,0.000000,5.168221,0,0.023752,0.389126,TCGA-B6-A1KC-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10814,5.312015,0.054779,4.956693,1.847115,0.827900,2.303225,8.427157,4.806525,4.081399,2.794083,...,0.00000,0.124593,0,0.000000,0.263756,2.721504,0,0.008487,0.422233,TCGA-W5-AA30-11
10815,7.072792,0.070252,6.705809,2.356285,1.864176,2.305241,2.342270,6.360315,4.059762,4.582773,...,1.15562,0.486148,0,0.000000,0.000000,3.773828,0,0.032806,1.390888,TCGA-W5-AA38-01
10816,2.132873,0.298893,6.025226,2.472904,1.310224,4.988394,3.466940,4.420954,3.570706,4.070312,...,0.00000,0.608525,0,1.080044,0.095182,2.120053,0,0.040261,1.388465,TCGA-W5-AA2H-01
10817,5.638529,0.000000,6.587676,3.354960,2.290601,2.730901,6.821506,6.342717,3.480162,3.630580,...,0.00000,0.492007,0,0.000000,0.054779,4.356369,0,0.132643,1.277747,TCGA-W5-AA2O-01


In [8]:
# add gene exp data to dataframe
df_multiomics = df_subset[['Sample', 'type']].merge(df_gene_exp_full, how = 'left', left_on = 'Sample', right_on = 'sample_id')
df_multiomics = df_multiomics.drop(columns = ['sample_id'])
df_multiomics.drop_duplicates(subset = ['Sample'], inplace= True)
df_multiomics.shape # (8467, 60662)
df_multiomics

,Sample,type,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,...,ENSG00000288661.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
0,TCGA-AQ-A0Y5-01,Primary Tumor,3.703721,0.463099,7.086452,4.051007,2.426989,2.322361,4.943860,6.539078,...,0,0.0,0.204767,0,0.000000,0.000000,4.773448,0,0.054779,1.243182
1,TCGA-C8-A1HI-01,Primary Tumor,5.269714,0.289834,6.623920,4.429529,2.521980,2.750221,4.703101,6.267481,...,0,0.0,0.299245,0,0.000000,0.000000,4.119074,0,0.036468,0.709467
2,TCGA-5L-AAT1-01,Primary Tumor,4.899388,0.363227,6.779602,4.121861,2.128788,3.447725,4.697963,5.676846,...,0,0.0,0.579470,0,0.000000,0.000000,3.656840,0,0.020911,1.778671
3,TCGA-BH-A0C0-01,Primary Tumor,5.711255,0.268195,7.578444,3.913043,3.320687,4.947152,4.318447,6.316104,...,0,0.0,0.228973,0,0.000000,0.000000,4.725218,0,0.098285,0.344033
4,TCGA-B6-A1KC-01,Primary Tumor,4.251984,0.435415,6.178436,3.558464,2.589548,1.508581,3.722149,5.224044,...,0,0.0,0.434028,0,0.000000,0.000000,5.168221,0,0.023752,0.389126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8541,TCGA-W5-AA31-01,Primary Tumor,7.573606,0.000000,6.811054,4.156073,2.804446,3.393155,7.001124,6.120097,...,0,0.0,0.713960,0,0.000000,0.044884,3.955954,0,0.055473,0.806530
8542,TCGA-W5-AA2U-01,Primary Tumor,6.734234,0.061707,6.014679,3.355200,1.857424,2.760157,5.085195,5.275465,...,0,0.0,0.445621,0,0.000000,0.000000,3.939161,0,0.075327,0.468531
8543,TCGA-ZU-A8S4-01,Primary Tumor,5.233106,0.000000,6.848891,2.474773,2.648627,4.104706,7.008132,7.073047,...,0,0.0,0.334225,0,0.000000,0.000000,3.441961,0,0.051442,1.265857
8544,TCGA-W5-AA2I-01,Primary Tumor,5.665532,0.000000,5.152155,2.413811,1.658417,1.514198,3.354396,5.825127,...,0,0.0,0.457752,0,0.000000,0.000000,3.449258,0,0.039419,1.239887


### miRNA data

In [9]:
df_mirna = pd.read_csv('for_yu_miRNA.csv')
df_mirna

,Unnamed: 0,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,...,hsa-mir-26b,hsa-mir-3116-1,hsa-mir-346,hsa-mir-4717,hsa-mir-548s,hsa-mir-5692c-1,hsa-mir-647,hsa-mir-6719,hsa-mir-6750,hsa-mir-6878
0,TCGA-BRCA1,12.815167,12.816051,12.855016,14.730158,11.169628,8.859833,9.505782,10.680853,10.722648,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TCGA-BRCA2,12.968838,12.972606,12.972485,14.497080,11.869264,7.239808,9.593766,11.532029,11.479348,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TCGA-BRCA3,13.263951,13.247779,13.270085,13.844852,10.904361,9.067461,10.097007,12.528655,12.536764,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TCGA-BRCA4,12.871879,12.870939,12.896584,14.935364,9.644680,7.849328,8.665824,9.653458,9.730741,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TCGA-BRCA5,13.865815,13.861453,13.867042,14.606950,11.237808,8.316985,10.444484,12.448795,12.471484,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11085,TCGA-CHOL41,12.397355,12.410310,12.409395,12.764282,9.716770,8.727966,10.744548,11.404266,11.454670,...,8.120079,0.000000,0.252878,0.000000,0.000000,0.000000,0.0,0.0,0.252878,0.0
11086,TCGA-CHOL42,12.271850,12.264286,12.282251,12.547766,9.808921,8.940528,10.275202,10.868141,10.869098,...,8.791045,0.235286,1.560000,0.000000,0.235286,0.000000,0.0,0.0,0.235286,0.0
11087,TCGA-CHOL43,13.402303,13.418145,13.407547,14.493636,10.488297,8.718217,9.983680,11.395059,11.392468,...,8.636674,0.326920,0.326920,0.000000,0.818032,0.000000,0.0,0.0,0.000000,0.0
11088,TCGA-CHOL44,12.914865,12.908972,12.913389,13.720097,9.832415,8.003953,10.085560,11.447787,11.492745,...,9.222545,0.142612,0.501421,0.142612,0.000000,0.142612,0.0,0.0,0.000000,0.0


In [10]:
for col in df_mirna.columns:
    if "hsa" not in col:
        print(col)

Unnamed: 0
type
Sample


In [11]:
df_mirna.drop(columns = ['Unnamed: 0', 'type',], inplace = True)
df_mirna

,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,...,hsa-mir-26b,hsa-mir-3116-1,hsa-mir-346,hsa-mir-4717,hsa-mir-548s,hsa-mir-5692c-1,hsa-mir-647,hsa-mir-6719,hsa-mir-6750,hsa-mir-6878
0,12.815167,12.816051,12.855016,14.730158,11.169628,8.859833,9.505782,10.680853,10.722648,8.288114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12.968838,12.972606,12.972485,14.497080,11.869264,7.239808,9.593766,11.532029,11.479348,8.007486,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13.263951,13.247779,13.270085,13.844852,10.904361,9.067461,10.097007,12.528655,12.536764,9.762910,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12.871879,12.870939,12.896584,14.935364,9.644680,7.849328,8.665824,9.653458,9.730741,8.959749,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13.865815,13.861453,13.867042,14.606950,11.237808,8.316985,10.444484,12.448795,12.471484,8.850126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11085,12.397355,12.410310,12.409395,12.764282,9.716770,8.727966,10.744548,11.404266,11.454670,8.626717,...,8.120079,0.000000,0.252878,0.000000,0.000000,0.000000,0.0,0.0,0.252878,0.0
11086,12.271850,12.264286,12.282251,12.547766,9.808921,8.940528,10.275202,10.868141,10.869098,8.997139,...,8.791045,0.235286,1.560000,0.000000,0.235286,0.000000,0.0,0.0,0.235286,0.0
11087,13.402303,13.418145,13.407547,14.493636,10.488297,8.718217,9.983680,11.395059,11.392468,9.127377,...,8.636674,0.326920,0.326920,0.000000,0.818032,0.000000,0.0,0.0,0.000000,0.0
11088,12.914865,12.908972,12.913389,13.720097,9.832415,8.003953,10.085560,11.447787,11.492745,8.792510,...,9.222545,0.142612,0.501421,0.142612,0.000000,0.142612,0.0,0.0,0.000000,0.0


In [12]:
# extract sample-level identifier (substring of the full barcode/sample ID)
df_mirna['sample_id'] = df_mirna.Sample.str.split('-', n=4).str[:4].str.join('-').str[:-1]
df_mirna = df_mirna.drop(columns = ['Sample'])
df_mirna

,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,...,hsa-mir-3116-1,hsa-mir-346,hsa-mir-4717,hsa-mir-548s,hsa-mir-5692c-1,hsa-mir-647,hsa-mir-6719,hsa-mir-6750,hsa-mir-6878,sample_id
0,12.815167,12.816051,12.855016,14.730158,11.169628,8.859833,9.505782,10.680853,10.722648,8.288114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-D8-A146-01
1,12.968838,12.972606,12.972485,14.497080,11.869264,7.239808,9.593766,11.532029,11.479348,8.007486,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-AQ-A0Y5-01
2,13.263951,13.247779,13.270085,13.844852,10.904361,9.067461,10.097007,12.528655,12.536764,9.762910,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-C8-A1HI-01
3,12.871879,12.870939,12.896584,14.935364,9.644680,7.849328,8.665824,9.653458,9.730741,8.959749,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-A7-A0CD-01
4,13.865815,13.861453,13.867042,14.606950,11.237808,8.316985,10.444484,12.448795,12.471484,8.850126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-5L-AAT1-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11085,12.397355,12.410310,12.409395,12.764282,9.716770,8.727966,10.744548,11.404266,11.454670,8.626717,...,0.000000,0.252878,0.000000,0.000000,0.000000,0.0,0.0,0.252878,0.0,TCGA-W5-AA31-01
11086,12.271850,12.264286,12.282251,12.547766,9.808921,8.940528,10.275202,10.868141,10.869098,8.997139,...,0.235286,1.560000,0.000000,0.235286,0.000000,0.0,0.0,0.235286,0.0,TCGA-W5-AA2U-01
11087,13.402303,13.418145,13.407547,14.493636,10.488297,8.718217,9.983680,11.395059,11.392468,9.127377,...,0.326920,0.326920,0.000000,0.818032,0.000000,0.0,0.0,0.000000,0.0,TCGA-ZU-A8S4-01
11088,12.914865,12.908972,12.913389,13.720097,9.832415,8.003953,10.085560,11.447787,11.492745,8.792510,...,0.142612,0.501421,0.142612,0.000000,0.142612,0.0,0.0,0.000000,0.0,TCGA-W5-AA2I-01


In [13]:
# add miRNA data to the multiomics dataframe
df_multiomics = df_multiomics.merge(df_mirna, how = 'left', left_on = 'Sample', right_on = 'sample_id')
df_multiomics = df_multiomics.drop(columns = ['sample_id'])
df_multiomics.drop_duplicates(subset = ['Sample'], inplace= True)
df_multiomics.shape # (8467, 62543)

(8467, 62543)

In [14]:
df_multiomics.value_counts('type') # 8099 primary tumor, 368 solid tissue normal

type
Primary Tumor          8099
Solid Tissue Normal     368
Name: count, dtype: int64

## 1(b). Add methylomics to dataframe with miRNA and gene expression
We will add a subset of columns from the methylomics data as we were unable to add all 450k methylation columns into our multiomics model.
Later, when we are splitting the data to build and test our model, we will follow the same train/test/val split that we used to obtain a subset of methylation columns to prevent any information leakage.

### Methylation data: distilled dataset with 593 features only

In [15]:
df_meth = pd.read_csv("TCGA_meth_593cols.csv", index_col=0)
df_meth

,cg00657094,cg00889627,cg00026222,cg00028507,cg09509365,cg00000957,cg07319375,cg00547365,cg05814135,cg00101629,...,cg26914705,cg07679679,cg00058879,cg20957542,cg00921237,cg27166177,cg05729745,cg25081912,cg00062282,sample_id
0,0.411621,0.621735,0.040575,0.161868,0.281257,0.777497,0.098279,0.617392,0.745925,0.325123,...,0.052327,0.380909,0.663547,0.747777,0.211242,0.114195,0.174401,0.816961,0.424337,TCGA-NF-A4X2-01
1,0.545119,0.340770,0.176231,0.331660,0.863144,0.844783,0.073168,0.545142,0.757941,0.861854,...,0.059471,0.335478,0.807092,0.636874,0.223941,0.125103,0.202899,0.848802,0.276662,TCGA-NG-A4VU-01
2,0.278215,0.400863,0.045977,0.267730,0.417079,0.688139,0.111705,0.649042,0.630707,0.856070,...,0.069232,0.557395,0.119348,0.588657,0.271522,0.152596,0.335041,0.846619,0.497601,TCGA-N8-A56S-01
3,0.292703,0.973566,0.056402,0.459386,0.781441,0.830256,0.080549,0.578502,0.838083,0.868675,...,0.046855,0.838569,0.835077,0.750678,0.320655,0.141215,0.150263,0.859772,0.483871,TCGA-N8-A4PL-01
4,0.583085,0.724319,0.024996,0.324168,0.720580,0.708335,0.099528,0.554873,0.686611,0.910438,...,0.044736,0.435550,0.112859,0.721100,0.684331,0.163656,0.317459,0.869584,0.282028,TCGA-N5-A4RO-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9781,0.469836,0.963279,0.033784,0.351272,0.880702,0.907370,0.143262,0.776855,0.946176,0.637099,...,0.067650,0.691867,0.096245,0.869565,0.296224,0.337493,0.308202,0.882717,0.614417,TCGA-KO-8403-01
9782,0.568090,0.974412,0.035681,0.330773,0.881204,0.934874,0.136689,0.810328,0.928536,0.943398,...,0.041640,0.508179,0.188961,0.860149,0.602053,0.199547,0.277133,0.924013,0.644224,TCGA-KN-8434-01
9783,0.885347,0.966679,0.034579,0.779984,0.849670,0.886904,0.329963,0.774311,0.740426,0.743007,...,0.069921,0.714920,0.113606,0.612634,0.877271,0.666925,0.646401,0.912015,0.487964,TCGA-KO-8405-01
9784,0.962767,0.985107,0.058243,0.146255,0.904303,0.919828,0.114765,0.914389,0.959818,0.933331,...,0.060348,0.628638,0.059546,0.889165,0.196848,0.148803,0.364181,0.907754,0.773497,TCGA-KN-8436-01


In [16]:
##
df_multiomics = df_multiomics.merge(df_meth, how = 'inner', left_on = 'Sample', right_on = 'sample_id')
df_multiomics = df_multiomics.drop(columns = ['sample_id'])

df_multiomics.drop_duplicates(subset = ['Sample'], inplace= True)

print(f"Multiomics (miRNA + gene expression + methylation) dataframe shape: {df_multiomics.shape}") # (8467, 63136)

Multiomics (miRNA + gene expression + methylation) dataframe shape: (8467, 63136)


# 2. Load External Validation file 
`df_external_validation`

In [17]:
start_time = time.time()

## External validation files with 3 omics (includes only 593 methylation features as above)
df_external_validation = pd.read_csv('external_validation/mirna_AND_gene_exp_AND_methylation_UPDATED_63138.csv.gz', 
                                     compression='gzip', sep=',', index_col=0)

print((time.time() - start_time) / 60)
print(df_external_validation.shape)
# df_external_validation (1842, 63138)

1.8570780158042908
(1842, 63138)


# 3. Preparing our data for modeling: Defining the Train/Val/Test sets
We will use same train and hold-out sets as the ones we used to obtain the top methylation features

### 3(a). Training set from TCGA (get the sample names from all datasets in `train_short` and `val_short` and merge)

In [18]:
train_dirname = 'meth_imp_cols/450k_with_ids/train_short/'
train_files = [f for f in os.listdir(train_dirname) if ('meth_450_' in f) and ('train.csv' in f)]

val_dirname = 'meth_imp_cols/450k_with_ids/val_short/'
val_files = [f for f in os.listdir(val_dirname) if ('meth_450_' in f) and ('val.csv' in f)]

files = train_files + val_files
print(len(files))
files

32


['meth_450_BLCA_train.csv',
 'meth_450_UCEC_train.csv',
 'meth_450_LUSC_train.csv',
 'meth_450_THCA_train.csv',
 'meth_450_PAAD_train.csv',
 'meth_450_CHOL_train.csv',
 'meth_450_BRCA_train.csv',
 'meth_450_READ_train.csv',
 'meth_450_KIRP_train.csv',
 'meth_450_ESCA_train.csv',
 'meth_450_LUAD_train.csv',
 'meth_450_COAD_train.csv',
 'meth_450_HNSC_train.csv',
 'meth_450_LIHC_train.csv',
 'meth_450_KIRC_train.csv',
 'meth_450_PRAD_train.csv',
 'meth_450_UCEC_val.csv',
 'meth_450_ESCA_val.csv',
 'meth_450_HNSC_val.csv',
 'meth_450_PRAD_val.csv',
 'meth_450_THCA_val.csv',
 'meth_450_KIRC_val.csv',
 'meth_450_LIHC_val.csv',
 'meth_450_BRCA_val.csv',
 'meth_450_BLCA_val.csv',
 'meth_450_READ_val.csv',
 'meth_450_COAD_val.csv',
 'meth_450_KIRP_val.csv',
 'meth_450_CHOL_val.csv',
 'meth_450_LUSC_val.csv',
 'meth_450_LUAD_val.csv',
 'meth_450_PAAD_val.csv']

In [19]:
samples = pd.DataFrame(columns = ['sample'])
for f_index, f in enumerate(files):
    print(f_index, f)
    
    if '_val.csv' in f:
        dirname = val_dirname
    elif '_train.csv' in f:
        dirname = train_dirname
    temp = pd.read_csv(dirname + f, usecols = ['sample'])
    samples = pd.concat([samples, temp])
    samples.reset_index(drop=True, inplace=True)
samples

0 meth_450_BLCA_train.csv
1 meth_450_UCEC_train.csv
2 meth_450_LUSC_train.csv
3 meth_450_THCA_train.csv
4 meth_450_PAAD_train.csv
5 meth_450_CHOL_train.csv
6 meth_450_BRCA_train.csv
7 meth_450_READ_train.csv
8 meth_450_KIRP_train.csv
9 meth_450_ESCA_train.csv
10 meth_450_LUAD_train.csv
11 meth_450_COAD_train.csv
12 meth_450_HNSC_train.csv
13 meth_450_LIHC_train.csv
14 meth_450_KIRC_train.csv
15 meth_450_PRAD_train.csv
16 meth_450_UCEC_val.csv
17 meth_450_ESCA_val.csv
18 meth_450_HNSC_val.csv
19 meth_450_PRAD_val.csv
20 meth_450_THCA_val.csv
21 meth_450_KIRC_val.csv
22 meth_450_LIHC_val.csv
23 meth_450_BRCA_val.csv
24 meth_450_BLCA_val.csv
25 meth_450_READ_val.csv
26 meth_450_COAD_val.csv
27 meth_450_KIRP_val.csv
28 meth_450_CHOL_val.csv
29 meth_450_LUSC_val.csv
30 meth_450_LUAD_val.csv
31 meth_450_PAAD_val.csv


,sample
0,TCGA-CU-A3KJ-01A-11D-A21B-05
1,TCGA-C4-A0F7-01A-11D-A10W-05
2,TCGA-K4-A3WU-01B-11D-A23O-05
3,TCGA-DK-A3IU-01A-11D-A211-05
4,TCGA-XF-A9SU-01A-31D-A392-05
...,...
5210,TCGA-FZ-5924-11A-01D-1744-05
5211,TCGA-XD-AAUL-01A-21D-A398-05
5212,TCGA-IB-7887-01A-11D-2157-05
5213,TCGA-HZ-A77O-01A-11D-A33U-05


In [20]:
# extract sample-level identifiers (substring of full sample IDs)
samples['sample_id'] = samples['sample'].str.split('-', n=4).str[:4].str.join('-').str[:-1]
samples = samples.drop(columns = ['sample'])
samples

,sample_id
0,TCGA-CU-A3KJ-01
1,TCGA-C4-A0F7-01
2,TCGA-K4-A3WU-01
3,TCGA-DK-A3IU-01
4,TCGA-XF-A9SU-01
...,...
5210,TCGA-FZ-5924-11
5211,TCGA-XD-AAUL-01
5212,TCGA-IB-7887-01
5213,TCGA-HZ-A77O-01


In [21]:
df = df_multiomics.copy()
df['target'] =  (df['type'] == 'Primary Tumor').astype(int)
df = df.drop(columns=['type'])

train_df = samples.merge(df, how = 'inner', left_on='sample_id', right_on='Sample')

# separate features and target
X_train_all = train_df.drop(columns=['Sample', 'target'])
y_train = train_df['target']

print(X_train_all.shape) # 63135 (= 63134 features columns + 1 sample id column)

(4863, 63135)


### 3(b). Hold-out test set from TCGA (get the sample names from all datasets in `test_short` and merge)

In [22]:
test_dirname = 'meth_imp_cols/450k_with_ids/test_short/'
files = [f for f in os.listdir(test_dirname) if ('meth_450_' in f) and ('test.csv' in f)]

print(len(files))
files

16


['meth_450_PRAD_test.csv',
 'meth_450_LUAD_test.csv',
 'meth_450_LUSC_test.csv',
 'meth_450_KIRC_test.csv',
 'meth_450_KIRP_test.csv',
 'meth_450_LIHC_test.csv',
 'meth_450_HNSC_test.csv',
 'meth_450_READ_test.csv',
 'meth_450_CHOL_test.csv',
 'meth_450_BLCA_test.csv',
 'meth_450_PAAD_test.csv',
 'meth_450_UCEC_test.csv',
 'meth_450_BRCA_test.csv',
 'meth_450_ESCA_test.csv',
 'meth_450_THCA_test.csv',
 'meth_450_COAD_test.csv']

In [23]:
samples = pd.DataFrame(columns = ['sample'])
for f_index, f in enumerate(files):
    print(f_index, f)
    if '_test.csv' in f:
        dirname = test_dirname
    temp = pd.read_csv(dirname + f, usecols = ['sample'])
    samples = pd.concat([samples, temp])
    samples.reset_index(drop=True, inplace=True)
samples

0 meth_450_PRAD_test.csv
1 meth_450_LUAD_test.csv
2 meth_450_LUSC_test.csv
3 meth_450_KIRC_test.csv
4 meth_450_KIRP_test.csv
5 meth_450_LIHC_test.csv
6 meth_450_HNSC_test.csv
7 meth_450_READ_test.csv
8 meth_450_CHOL_test.csv
9 meth_450_BLCA_test.csv
10 meth_450_PAAD_test.csv
11 meth_450_UCEC_test.csv
12 meth_450_BRCA_test.csv
13 meth_450_ESCA_test.csv
14 meth_450_THCA_test.csv
15 meth_450_COAD_test.csv


,sample
0,TCGA-G9-6338-01A-12D-1963-05
1,TCGA-EJ-7781-11A-01D-2116-05
2,TCGA-HC-A8D1-01A-11D-A365-05
3,TCGA-EJ-7315-01A-31D-2116-05
4,TCGA-VP-A878-01A-31D-A34V-05
...,...
1329,TCGA-AY-A71X-01A-12D-A36Y-05
1330,TCGA-CM-5860-01A-01D-1651-05
1331,TCGA-CM-6674-01A-11D-1837-05
1332,TCGA-G4-6294-01A-11D-1772-05


In [24]:
samples['sample_id'] = samples['sample'].str.split('-', n=4).str[:4].str.join('-').str[:-1]
samples = samples.drop(columns = ['sample'])
samples

,sample_id
0,TCGA-G9-6338-01
1,TCGA-EJ-7781-11
2,TCGA-HC-A8D1-01
3,TCGA-EJ-7315-01
4,TCGA-VP-A878-01
...,...
1329,TCGA-AY-A71X-01
1330,TCGA-CM-5860-01
1331,TCGA-CM-6674-01
1332,TCGA-G4-6294-01


In [25]:
df = df_multiomics.copy()
df['target'] =  (df['type'] == 'Primary Tumor').astype(int)
df = df.drop(columns=['type'])

test_df = samples.merge(df, how = 'inner', left_on='sample_id', right_on='Sample')

# separate features and target
X_test_all = test_df.drop(columns=['Sample', 'target'])
y_test = test_df['target']

print(X_test_all.shape) # 63135 (= 63134 features columns + 1 sample id column)

(1239, 63135)


### 3(c). External test set

In [26]:
# external test set
df_ext_test = df_external_validation.copy()
df_ext_test['target'] = (df_ext_test['Type'] == 'Primary Tumor').astype(int)
df_ext_test = df_ext_test.drop(columns=['Type'])

# separate features and target
X_ext_test_all = df_ext_test.drop(columns=['target'])
y_ext_test = df_ext_test['target']
print(X_ext_test_all.shape) # 63137 (= 63134 features + 1 sample id + 1 site + 1 stage)

(1842, 63137)


## Summary of part 3.

In [27]:
print(X_train_all.shape, y_train.shape)
print(X_test_all.shape, y_test.shape)
print(X_ext_test_all.shape, y_ext_test.shape)

(4863, 63135) (4863,)
(1239, 63135) (1239,)
(1842, 63137) (1842,)


# 4. Modeling
time to train and evaluate!


## 4.1. Single omics type: Methylation

In [28]:
meth_columns = [col for col in df_multiomics.columns if 'cg' in col]  # all methylation columns (after pre-selection)
# meth_columns = list(pd.read_csv('results/features/features_meth_top60.csv', usecols = ['Feature']).Feature) # selected features

print(len(meth_columns))

X_train = X_train_all[meth_columns]
X_test = X_test_all[meth_columns]
X_ext_test = X_ext_test_all[meth_columns]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_ext_test.shape, y_ext_test.shape)

593
(4863, 593) (4863,)
(1239, 593) (1239,)
(1842, 593) (1842,)


In [29]:
start_time = time.time()

# Define the model
model = XGBClassifier(objective='binary:logistic', tree_method='gpu_hist', gpu_id=0)

# Define the hyperparameters for grid search
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [5, ],
    'learning_rate': [0.01, 0.02,],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1)

# Fit the model with cross-validation
grid_search.fit(X_train, y_train)

# Get the best model and hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print(f"Best hyperparameters: {best_params}")
print("========================================")

# Train the best model on the full training set
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate the AUC score
auc = roc_auc_score(y_test, y_pred_prob)
print(f"Test AUC: {auc:.4f}")
print("========================================")

# Print execution time
print(f"Time taken: {(time.time() - start_time)/60:.2f} minutes")

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:42:09] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:42:09] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:42:09] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:42:09] WARNING: 

Best hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.02, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}
Test AUC: 0.9913
Time taken: 0.41 minutes


In [30]:
y_ext_pred_prob = best_model.predict_proba(X_ext_test)[:, 1]  # Probabilities for the positive class

# Calculate the AUC score
auc = roc_auc_score(y_ext_test, y_ext_pred_prob)
print(f"Test AUC: {auc:.4f}")
print("========================================")

# Print execution time
print(f"Time taken: {(time.time() - start_time)/60:.2f} minutes")

Test AUC: 0.9813
Time taken: 0.41 minutes


In [31]:
# feature selection
num_top_features = 60
importance = best_model.feature_importances_

importance_df = pd.DataFrame({
    'Feature': X_test.columns,
    'Importance': importance
})

importance_df = importance_df.sort_values(by='Importance', ascending=False)

top_meth_cols = importance_df.head(num_top_features)
# top_meth_cols.to_csv(f'results/features/features_meth_top{num_top_features}.csv')
print(f'Saved to results/features/features_meth_top{num_top_features}.csv')

Saved to results/features/features_meth_top60.csv


## 4.2. Single omics type: gene expression

In [32]:
gene_exp_cols = [col for col in df_multiomics.columns if 'ENSG' in col] # all gene expression cols
# gene_exp_cols = list(pd.read_csv('results/features/features_gene_exp_top60.csv', usecols = ['Feature']).Feature) # selected features

print(len(gene_exp_cols))

X_train = X_train_all[gene_exp_cols]
X_test = X_test_all[gene_exp_cols]
X_ext_test = X_ext_test_all[gene_exp_cols]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_ext_test.shape, y_ext_test.shape)

60660
(4863, 60660) (4863,)
(1239, 60660) (1239,)
(1842, 60660) (1842,)


In [33]:
start_time = time.time()

# Define the model
model = XGBClassifier(objective='binary:logistic', tree_method='gpu_hist', gpu_id=0)

# Define the hyperparameters for grid search
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [5, ],
    'learning_rate': [0.01, 0.02,],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1)

# Fit the model with cross-validation
grid_search.fit(X_train, y_train)

# Get the best model and hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print(f"Best hyperparameters: {best_params}")
print("========================================")

# Train the best model on the full training set
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate the AUC score
auc = roc_auc_score(y_test, y_pred_prob)
print(f"Test AUC: {auc:.4f}")
print("========================================")

# Print execution time
print(f"Time taken: {(time.time() - start_time)/60:.2f} minutes")

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:43:46] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:43:46] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:43:47] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:43:47] WARNING: 

Best hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}
Test AUC: 0.9917
Time taken: 3.80 minutes


In [34]:
y_ext_pred_prob = best_model.predict_proba(X_ext_test)[:, 1]  # Probabilities for the positive class

# Calculate the AUC score
auc = roc_auc_score(y_ext_test, y_ext_pred_prob)
print(f"Test AUC: {auc:.4f}")
print("========================================")

# Print execution time
print(f"Time taken: {(time.time() - start_time)/60:.2f} minutes")

Test AUC: 0.9830
Time taken: 3.97 minutes


In [35]:
# feature selection
num_top_features = 60
importance = best_model.feature_importances_

importance_df = pd.DataFrame({
    'Feature': X_test.columns,
    'Importance': importance
})

importance_df = importance_df.sort_values(by='Importance', ascending=False)

top_gene_exp_cols = importance_df.head(num_top_features)
# top_gene_exp_cols.to_csv(f'results/features/features_gene_exp_top{num_top_features}.csv')
print(f'Saved to results/features/features_gene_exp_top{num_top_features}.csv')

Saved to results/features/features_gene_exp_top60.csv


## 4.3. Single omics type: miRNA

In [36]:
mirna_cols = [col for col in df_multiomics.columns if 'hsa' in col]  # all miRNA columns
# mirna_cols = list(pd.read_csv('results/features/features_mirna_top60.csv', usecols = ['Feature']).Feature) # selected features

print(len(mirna_cols))

X_train = X_train_all[mirna_cols]
X_test = X_test_all[mirna_cols]
X_ext_test = X_ext_test_all[mirna_cols]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_ext_test.shape, y_ext_test.shape)

1881
(4863, 1881) (4863,)
(1239, 1881) (1239,)
(1842, 1881) (1842,)


In [37]:
start_time = time.time()

# Define the model
model = XGBClassifier(objective='binary:logistic', tree_method='gpu_hist', gpu_id=0)

# Define the hyperparameters for grid search
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [5, ],
    'learning_rate': [0.01, 0.02,],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1)

# Fit the model with cross-validation
grid_search.fit(X_train, y_train)

# Get the best model and hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print(f"Best hyperparameters: {best_params}")
print("========================================")

# Train the best model on the full training set
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate the AUC score
auc = roc_auc_score(y_test, y_pred_prob)
print(f"Test AUC: {auc:.4f}")
print("========================================")

# Print execution time
print(f"Time taken: {(time.time() - start_time)/60:.2f} minutes")

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:46:26] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:46:26] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:46:26] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:46:26] WARNING: 

Best hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.02, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}
Test AUC: 0.9907
Time taken: 0.39 minutes


In [38]:
y_ext_pred_prob = best_model.predict_proba(X_ext_test)[:, 1]  # Probabilities for the positive class

# Calculate the AUC score
auc = roc_auc_score(y_ext_test, y_ext_pred_prob)
print(f"Test AUC: {auc:.4f}")
print("========================================")

# Print execution time
print(f"Time taken: {(time.time() - start_time)/60:.2f} minutes")

Test AUC: 0.9420
Time taken: 0.39 minutes


In [39]:
# feature selection
num_top_features = 60
importance = best_model.feature_importances_

importance_df = pd.DataFrame({
    'Feature': X_test.columns,
    'Importance': importance
})

importance_df = importance_df.sort_values(by='Importance', ascending=False)

top_mirna_cols = importance_df.head(num_top_features)
# top_mirna_cols.to_csv(f'results/features/features_mirna_top{num_top_features}.csv')
print(f'results/features/features_mirna_top{num_top_features}.csv')

results/features/features_mirna_top60.csv


## 4.4. Multiomics

In [40]:
# function to count number features per each omics included 
def count_features(df_column):
    methylation_count = 0
    gene_exp_count = 0
    miRNA_count = 0

    sample_count = 0
    site_count = 0
    stage_count = 0
    type_count = 0

    overall_count = 0

    for col in df_column:
        overall_count += 1
        if 'ENSG' in col:
            gene_exp_count += 1

        if 'hsa' in col:
            miRNA_count += 1

        if 'cg' in col:
            methylation_count += 1
    col_counts = pd.DataFrame()
    col_counts['col_type'] = ['Methylation', 'Gene_Expression', 'miRNA']
    col_counts['col_type_count'] = [methylation_count, gene_exp_count, miRNA_count]
    # print(overall_count)
    # print(sum(col_counts.col_type_count) == overall_count)
    return col_counts

In [41]:
# prevent overload
import gc
del model
gc.collect()

91

In [42]:
## 3 omics types (gene expression + miRNA + methylation)
selected_cols = [x for x in df_multiomics.columns if ('ENSG' in x) or ('hsa' in x) or ('cg' in x)] # include all features in all omics types 
# selected_cols = list(pd.read_csv('results/features/features_tri_omics_top1500_overall.csv', usecols = ['Feature']).Feature) # selected features 
# selected_cols = list(pd.read_csv('results/features/features_tri_omics_top500_per_prefix.csv', usecols = ['Feature']).Feature) # selected features 


## 2 omics types (gene expression + methylation)
# selected_cols = [x for x in df_multiomics.columns if ('ENSG' in x) or ('cg' in x)] # all features in the 2 selected omics types (gene expression + methylation)
# selected_cols = list(pd.read_csv('results/features/features_bi_omics_gene_and_meth_top1000_overall.csv', usecols = ['Feature']).Feature) # selected features 

print(len(selected_cols))

X_train = X_train_all[selected_cols]
X_test = X_test_all[selected_cols]
X_ext_test = X_ext_test_all[selected_cols]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_ext_test.shape, y_ext_test.shape)

63134
(4863, 63134) (4863,)
(1239, 63134) (1239,)
(1842, 63134) (1842,)


In [43]:
start_time = time.time()

# Define the model
model = XGBClassifier(objective='binary:logistic', tree_method='gpu_hist', gpu_id=0)

# Define the hyperparameters for grid search
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [5, ],
    'learning_rate': [0.01, 0.02, 0.05],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1)

# Fit the model with cross-validation
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:48:27] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:48:27] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:48:27] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
/home/gridsan/cwang1/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [13:48:27] WARNING: 

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None, gpu_id=0,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_r...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8],
                         'learning_rate': [0.01, 0.02, 0.05], 'max_depth': [5],
                         'n_estimators': [100, 300], 'subsample': [0.8]},
             scoring='roc_auc', verbose=1)

In [44]:
# Get the best model and hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print(f"Best hyperparameters: {best_params}")
print("========================================")

# Train the best model on the full training set
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate the AUC score
auc = roc_auc_score(y_test, y_pred_prob)
print(f"Test AUC: {auc:.4f}")
print("========================================")

# Print execution time
print(f"Time taken: {(time.time() - start_time)/60:.2f} minutes")

Best hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}
Test AUC: 0.9909
Time taken: 6.04 minutes


In [45]:
y_ext_pred_prob = best_model.predict_proba(X_ext_test)[:, 1]  # Probabilities for the positive class

# Calculate the AUC score
auc = roc_auc_score(y_ext_test, y_ext_pred_prob)
print(f"Test AUC: {auc:.4f}")
print("========================================")

print(f"Time taken: {(time.time() - start_time)/60:.2f} minutes")

Test AUC: 0.9805
Time taken: 6.21 minutes


In [46]:
count_features(X_train.columns)

,col_type,col_type_count
0,Methylation,593
1,Gene_Expression,60660
2,miRNA,1881


### 4.4.1. Feature Selection Approach 1 (Select top n overall)

In [47]:
importance = best_model.feature_importances_
num_top_features = 1500 

importance_df = pd.DataFrame({
    'Feature': X_test.columns,
    'Importance': importance
})

importance_df = importance_df.sort_values(by='Importance', ascending=False)

top_multiomics_cols = importance_df.head(num_top_features)

# top_multiomics_cols.to_csv(f'results/features/features_tri_omics_gene_and_meth_top{num_top_features}_overall.csv')
print(f"Saved to results/features/features_tri_omics_gene_and_meth_top{num_top_features}_overall.csv")

Saved to results/features/features_tri_omics_gene_and_meth_top1500_overall.csv


In [48]:
count_features(list(top_multiomics_cols.Feature.values))

,col_type,col_type_count
0,Methylation,86
1,Gene_Expression,1372
2,miRNA,42


### 4.4.2. Feature Selection Approach 2 (Balanced: Select top k per omics)

In [49]:
importance = best_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X_test.columns,
    'Importance': importance
})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
importance_df['prefix'] = importance_df['Feature'].str.extract(r'^(ENSG|cg|hsa)')
print(importance_df.shape)

n_features_per_omics = 500

top_features_per_prefix = importance_df.groupby('prefix').head(n_features_per_omics)

display(top_features_per_prefix)
display(count_features(list(top_features_per_prefix.Feature.values)))

# top_features_per_prefix.to_csv(f'results/features/features_tri_omics_top{n_features_per_omics}_per_prefix.csv')
print(f"Saved to results/features/features_tri_omics_top{n_features_per_omics}_per_prefix.csv")

(63134, 3)


,Feature,Importance,prefix
10641,ENSG00000161888.11,0.033914,ENSG
60942,hsa-mir-21,0.021350,hsa
3427,ENSG00000107562.16,0.019378,ENSG
6793,ENSG00000133800.9,0.017456,ENSG
62735,cg00808170,0.016653,cg
...,...,...,...
62872,cg00379708,0.000000,cg
62873,cg02910987,0.000000,cg
62875,cg00006081,0.000000,cg
62877,cg00932063,0.000000,cg


,col_type,col_type_count
0,Methylation,500
1,Gene_Expression,500
2,miRNA,500


Saved to results/features/features_tri_omics_top500_per_prefix.csv
